<a href="https://colab.research.google.com/github/96jayy/Jung-Minjae/blob/dacon/%EB%8C%80%EA%B5%AC%EA%B5%90%ED%86%B5%EC%82%AC%EA%B3%A0%EC%98%88%EC%B8%A1_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool

In [ ]:
train_org = pd.read_csv('/content/drive/MyDrive/dacon/daegu/train.csv').drop(columns='ID')
test_org = pd.read_csv('/content/drive/MyDrive/dacon/daegu/test.csv').drop(columns='ID')

In [ ]:
train_df = train_org.copy()
test_df = test_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다

# 해당 과정을 test_x에 대해서도 반복해줍니다
test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

In [ ]:
test_x = test_df
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [ ]:
train_x

In [ ]:
from pycaret.regression import *

In [ ]:
reg_experiment = setup(train, target='ECLO', session_id=42)

# 최적의 모델 찾기 (RMSLE를 성능 지표로 사용)
best_model = compare_models(fold=5, sort='RMSLE', n_select=1)

# CatBoost 모델 학습
catboost_model = create_model('catboost', fold=5)

# 앙상블 모델 구성

In [ ]:
ensemble_model = blend_models(estimator_list=[best_model, catboost_model])

In [ ]:
predictions = predict_model(ensemble_model, test_x)

In [ ]:
predictions['prediction_label']

In [ ]:
ss=pd.read_csv('/content/drive/MyDrive/dacon/daegu/sample_submission.csv')
ss['ECLO']=predictions['prediction_label']

In [ ]:
ss.to_csv('/content/drive/MyDrive/dacon/daegu/sub9.csv',index=False)

In [ ]:
light_df = pd.read_csv('/content/drive/MyDrive/dacon/daegu/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
#공백으로 구분된 네 개의 비공백 문자열을 찾음

light_df[['도시', '구', '동', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])

light_df = light_df.groupby(['도시', '구', '동']).sum().reset_index()
#도시, 구, 동 열을 기준으로 데이터를 그룹화 함.
#sum()을 해주는 이유는 각 데이터의 도시, 구, 동에 대한 보안등의 개수 합
light_df.reset_index(inplace=True, drop=True)
#reset_index(): 데이터프레임의 인덱스 초기화.

In [ ]:
child_area_df = pd.read_csv('/content/drive/MyDrive/dacon/daegu/대구 어린이 보호 구역 정보.csv', encoding='cp949')[['소재지지번주소']]
#중복된 값이 있으면 값이 겹칠 수 있으니 미리 중복 제거
child_area_df['School Zone'] = 1

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

child_area_df[['도시', '구', '동', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지'])

child_area_df = child_area_df.groupby(['도시', '구', '동']).sum().reset_index()
child_area_df.reset_index(inplace=True, drop=True)

In [ ]:
parking_df = pd.read_csv('/content/drive/MyDrive/dacon/daegu/대구 주차장 정보.csv', encoding='cp949')[['소재지지번주소', '급지구분']]
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])
#parking_df값들을 one_hot encoding으로 진행

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

parking_df[['도시', '구', '동', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])

parking_df = parking_df.groupby(['도시', '구', '동']).sum().reset_index()
parking_df.reset_index(inplace=True, drop=True)

In [ ]:
train_df = pd.merge(train_df, light_df, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, child_area_df, how='left', on=['도시', '구', '동'])
train_df = pd.merge(train_df, parking_df, how='left', on=['도시', '구', '동'])

test_df = pd.merge(test_df, light_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, child_area_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, parking_df, how='left', on=['도시', '구', '동'])

In [ ]:
train_df=train_df.fillna(0)
test_df=test_df.fillna(0)

In [ ]:
test_x = test_df #column ID열 제거하고 붙여넣기
train_x = train_df[test_x.columns].copy() #test.columns 값만 넣기
train_y = train_df['ECLO'].copy()

In [ ]:
train_x=train_x.drop('도시',axis=1)
test_x=test_x.drop('도시',axis=1)

In [ ]:
train=pd.concat([train_x,train_y],axis=1)

In [ ]:
reg_experiment = setup(train, target='ECLO', session_id=42)

# 최적의 모델 찾기 (RMSLE를 성능 지표로 사용)
catboost_model = create_model('catboost', fold=5)
best_model = compare_models(fold=5, sort='RMSLE', n_select=1)


# 앙상블 모델 구성

In [ ]:
tuned_model=tune_model(best_model)

In [ ]:
ensemble_model = blend_models(estimator_list=[tuned_model, catboost_model])

In [ ]:
predict=predict_model(ensemble_model,test_x)
predict.head()

In [ ]:
ss=pd.read_csv('/content/drive/MyDrive/dacon/daegu/sample_submission.csv')
ss['ECLO']=predict['prediction_label']

In [ ]:
ss.to_csv('/content/drive/MyDrive/dacon/daegu/sub10.csv',index=False)